In [18]:
from datasets import load_dataset, concatenate_datasets
import json
import torch
from datasets.utils.logging import disable_progress_bar,enable_progress_bar

dataset_name = "super_glue"  # The MMLU dataset name in Hugging Face Datasets
dataset = load_dataset(dataset_name, name='multirc')#["train"]
# print(dataset)

labels = dataset["train"].features["label"]
# def create_label_str(batch):
#     return {"label_str": labels.int2str(batch["label"])}
def create_label_str(example):
    return {"label_str": "True" if example["label"]== 1 else "False"}
# print(labels)


In [7]:
d1 = dataset["train"].filter(lambda example: example['idx']['question']==0)
for q in d1:
    print(q)

{'paragraph': 'While this process moved along, diplomacy continued its rounds. Direct pressure on the Taliban had proved unsuccessful. As one NSC staff note put it, "Under the Taliban, Afghanistan is not so much a state sponsor of terrorism as it is a state sponsored by terrorists." In early 2000, the United States began a high-level effort to persuade Pakistan to use its influence over the Taliban. In January 2000, Assistant Secretary of State Karl Inderfurth and the State Department\'s counterterrorism coordinator, Michael Sheehan, met with General Musharraf in Islamabad, dangling before him the possibility of a presidential visit in March as a reward for Pakistani cooperation. Such a visit was coveted by Musharraf, partly as a sign of his government\'s legitimacy. He told the two envoys that he would meet with Mullah Omar and press him on  Bin Laden. They left, however, reporting to Washington that Pakistan was unlikely in fact to do anything," given what it sees as the benefits of 

In [20]:
# data_train = dataset["train"].map(create_label_str)   
# data_test = dataset["validation"].map(create_label_str)   
# # print(d1)
# # dd = d1.select(range(5)).shuffle()

d1 = dataset["train"].map(create_label_str)   
d2 = dataset["validation"].map(create_label_str)   

d_all = concatenate_datasets([d1,d2])
train_size = 1000
data_train = d_all.select(range(train_size))
data_test = d_all.select(range(train_size,d_all.num_rows))

print(data_train.num_rows, data_test.num_rows)

# prepare examples
example_prompt = []
for ex in range(6):
    dd = data_train.filter(lambda example: example['idx']['question']==ex)
    prompt = "Paragraph: " + dd[0]['paragraph'] + '\n'
    prompt += "Question: " + dd[0]['question'] + '\n'
    # prompt += "Candidate answers:\n"
    # prompt += "Statement: "
    prompt += "Determine whether the following items are True or False. \n"
    for idx in range(dd.num_rows):
    # for idx in [0]:
        prompt += dd[idx]['answer'] + '\n'
    prompt += "Answer:"
    if ex< 4:
        for idx in range(dd.num_rows-1):
        # for idx in [0]:
            prompt += dd[idx]['label_str'] # + ','
        prompt += dd[idx]['label_str'] #+ '\n\n'
    example_prompt.append(prompt)

1000 31091


In [22]:

def json_writer(JSON_PATH, num_qs, n_shot_example,seed=10,space=False):
    qlist = {}
    
    with open(JSON_PATH, 'w') as json_file:

        for ind in range(num_qs):
            idx_ex_shuffle = torch.randperm(5)
            init_prompt = ''
            for i in range(n_shot_example):
                init_prompt +=  example_prompt[idx_ex_shuffle[i]] + '\n\n'

            dd = data_test.filter(lambda example: example['idx']['question']==ind)

            init_prompt += "Paragraph: " + dd[0]['paragraph'] + '\n'
            prompt = init_prompt
            for idx_q in range(dd.num_rows):
                prompt += "Question: " + dd[idx_q]['question'] + '\n'
                prompt += "Response: " + dd[idx_q]['answer'] + '\n'
                
                qlist['question'] = ind
                qlist['input'] = prompt
                qlist['target_str'] = dd[idx_q]['label_str']
                qlist['target'] = dd[idx_q]['label']

                 # Serializing json
                json_file.write(json.dumps(qlist) + '\n')
        # print("Done!")

In [23]:
disable_progress_bar()

# print(example_idx)
n_example = 10#data_test.num_rows
n_shot_vals = [2]
seed = 123
for n_shot in range(6):
# for n_shot in n_shot_vals:
    print(f"{n_shot}-shot example is being generated...")
    fname_json = f"multirc-data/{n_shot}_shot_examples.json"
    # fname_json = f"boolq-data/{n_shot}_shot_examples_small.json"
    # JSON_eval = 'sample_eval.json'
    json_writer(fname_json, n_example, n_shot, seed=seed, space= False)
    # json_writer(JSON_eval,example_idx[(n_shot_example+1)*n_test_ex:])

enable_progress_bar()


0-shot example is being generated...


KeyError: 'passage'

In [4]:
# fname_json = f"mmlu-data/ex_no_space.json"
n_shot = 0
d1 = load_dataset("json", data_files={
        'dev' : f"boolq-data/{n_shot}_shot_examples.json"
    })
# print(d1['dev'])
# print(d1['dev'].filter(lambda example: example["subject"] == subject))
# d2 = d1['dev'].filter(lambda example: example["subject"] == subject)
# # print(d2['input'][0][-1])
# # print("---------")
print(d1['dev'][2]['input'])

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1433.95it/s]
Generating dev split: 3270 examples [00:00, 158649.11 examples/s]

Passage: Phantom pain -- Phantom pain sensations are described as perceptions that an individual experiences relating to a limb or an organ that is not physically part of the body. Limb loss is a result of either removal by amputation or congenital limb deficiency. However, phantom limb sensations can also occur following nerve avulsion or spinal cord injury.
Question: is pain experienced in a missing body part or paralyzed area
Answer:


In [11]:
from datasets import DatasetDict, Dataset
from collections import defaultdict

fname_dict = {}
for n_shot in range(6):
    fname_dict[f"{n_shot}_shot"] = f"boolq-data/{n_shot}_shot_examples.json"
# data_all = defaultdict(DatasetDict)
d_upload = load_dataset("json", data_files=fname_dict)
    # data_all[f"n_shot = {n_shot}"] = d1['dev']


Extracting data files: 100%|██████████| 6/6 [00:00<00:00, 2024.77it/s]
Generating 0_shot split: 100 examples [00:00, 38178.63 examples/s]
Generating 1_shot split: 100 examples [00:00, 46644.84 examples/s]
Generating 2_shot split: 100 examples [00:00, 24292.27 examples/s]
Generating 3_shot split: 100 examples [00:00, 30995.45 examples/s]
Generating 4_shot split: 100 examples [00:00, 21987.33 examples/s]
Generating 5_shot split: 100 examples [00:00, 16446.32 examples/s]


In [12]:
# print(Dataset.from_dict(data_all))
# Dataset.from_dict(data_all).push_to_hub("hassansh/boolq_n_shot")
d_upload.push_to_hub("hassansh/boolq_n_shot")


Uploading the dataset shards: 100%|██████████| 1/1 [00:09<00:00,  9.19s/it]
README.md: 100%|██████████| 1.32k/1.32k [00:00<00:00, 3.12MB/s]


In [ ]:
print(d_upload['5_shot'][2]['input'])

Passage: The Elder Scrolls Online -- As with other games in The Elder Scrolls series, the game is set on the continent of Tamriel. The events of the game occur a millennium before those of The Elder Scrolls V: Skyrim and around 800 years before The Elder Scrolls III: Morrowind and The Elder Scrolls IV: Oblivion. It has a broadly similar structure to Skyrim, with two separate conflicts progressing at the same time, one with the fate of the world in the balance, and one where the prize is supreme power on Tamriel. In The Elder Scrolls Online, the first struggle is against the Daedric Prince Molag Bal, who is attempting to meld the plane of Mundus with his realm of Coldharbour, and the second is to capture the vacant imperial throne, contested by three alliances of the mortal races. The player character has been sacrificed to Molag Bal, and Molag Bal has stolen their soul, the recovery of which is the primary game objective.
Question: is elder scrolls online the same as skyrim
Answer:Fals

In [10]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 9427
    })
    validation: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 3270
    })
    test: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 3245
    })
})
